In [ ]:
import os
from os.path import dirname
import collections
import cv2
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Получение треков

In [ ]:
!git clone --recurse-submodules https://github.com/rizomus/yolov8_tracking.git
!cp yolov8_tracking/requirements.txt /content/
!pip install -r requirements.txt  

Cloning into 'yolov8_tracking'...
remote: Enumerating objects: 4022, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 4022 (delta 22), reused 0 (delta 0), pack-reused 3983
Receiving objects: 100% (4022/4022), 146.66 MiB | 34.70 MiB/s, done.
Resolving deltas: 100% (2203/2203), done.
Submodule 'yolov8' (https://github.com/ultralytics/ultralytics) registered for path 'yolov8'
Cloning into '/content/yolov8_tracking/yolov8'...
remote: Enumerating objects: 7674, done.        
remote: Counting objects: 100% (313/313), done.        
remote: Compressing objects: 100% (212/212), done.        
remote: Total 7674 (delta 187), reused 191 (delta 101), pack-reused 7361        
Receiving objects: 100% (7674/7674), 5.21 MiB | 6.43 MiB/s, done.
Resolving deltas: 100% (5317/5317), done.
Submodule path 'yolov8': checked out '15b3b0365ab2f12993a58985f3cb7f2137409a0c'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/cola

In [ ]:
!rm -r /content/yolov8_tracking/runs

rm: cannot remove '/content/yolov8_tracking/runs': No such file or directory


In [ ]:
test_vid_dir = '/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test'

In [ ]:
for file in os.listdir(test_vid_dir):
    filepath = os.path.join(test_vid_dir, file)
    string = 'python3 yolov8_tracking/track.py --yolo-weights /content/drive/MyDrive/Hardhats_vests/yolov8s.pt --tracking-method ocsort --source ' + filepath + ' --save-txt --save-conf --save-trajectories --save-vid'
    print(filepath)
    os.system(string)


/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/1.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/2.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/3.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/5.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/4.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/6.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/7.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/8.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/9.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/10.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/11.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/12.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/13.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/14.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/15.mp4
/content/drive/MyDrive/Hardhats_vests/dataset-v1.1/test/16.mp4
/

In [ ]:
# !cp -r /content/yolov8_tracking/runs/track /content/drive/MyDrive/

In [ ]:
# Путь в папку с результатами трекинга
directory = '/content/yolov8_tracking/runs/track'

# Задаем функции и подсчитываем людей, каски и жилеты.

In [ ]:
def get_f_number(filepath):
    filename = os.path.basename(filepath)
    file_number = os.path.splitext(filename)[0]
    return file_number

In [ ]:
files_dict = {}
for address, dirs, files in os.walk(directory):
    for filepath in files:
        ext = os.path.splitext(filepath)[1]
        file_number = int(get_f_number(filepath))
        if ext == '.txt':
            files_dict[file_number] = os.path.join(address, filepath)
od = collections.OrderedDict(sorted(files_dict.items()))

In [ ]:
def intersection_ratio(x1, y1, w1, h1, x2, y2, w2, h2, wear):         # Доля площади бб шлема, перекрываемая бб человека (1 - human, 2 - helmet)
    left_x = max(x1, x2)
    upper_y = max(y1, y2)
    right_x = min(x1 + w1, x2 + w2)
    bottom_y = min(y1 + h1, y2 + h2)

    helmet_area = w2 * h2
    intersection_area = (right_x - left_x) * (bottom_y - upper_y)

    if (wear=='helmet' and abs(y1 - y2) < res / 10) or wear=='uniform':                     # если шлем далеко от верхней границы бб человека (res - разрешение видео по вертикали - глобальная переменная)
        return intersection_area / helmet_area
    else:
        return 0

In [ ]:
def check_work_wear(wear):
    
        '''
        Проверка касок/жилетов.
        Проверка происходит только для посчитаных людей (actual_human_id).
        Для каждого человека считается процент перекрытия им каждого шлема, находящегося с ним в кадре и вычисляется среднее по всем кадрам. 
        Если значение для данного шлема выше 0,7 и количество совместных кадров > 15 - считаем, что человек в шлеме. 
        '''

        # all_classes_tracks, actual_human_id, human_tracks, log_book, frame_dict - глобальные переменные

        if wear == 'helmet': cls = 1 
        elif wear == 'uniform': cls = 2 

        wear_mask = all_classes_tracks['Class']==cls
        wear_tracks = all_classes_tracks[wear_mask]                 # треки касок или жилетов
        wear_id_list = list(wear_tracks['id'].unique())

        personal_human_tracks = {}
        for id in actual_human_id:
            id_track = human_tracks[human_tracks['id']==id]
            personal_human_tracks[id] = id_track                    # треки людей

        pairs = pd.DataFrame(columns=wear_id_list, index=actual_human_id)                 # Таблица учёта степени перекрытия шлемов человеками
        for i in actual_human_id:                                                         # Строки - id человеков
            for c in wear_id_list:                                                        # Столбцы - id шлемов
                pairs.loc[i][c] = []                                                      # В ячейке будет список значений intersection_ratio для последовательности кадров

        for human_track in personal_human_tracks.values():          # для каждой пары человек-шлем в одном кадре считаем intersection_ratio и сохраняем в таблице список значений
            for i in human_track.index:
                frame, human_id, human_x, human_y, human_w, human_h = human_track.loc[i]['Frame':'height']
                for ind in frame_dict[frame]:                                                                       # по всем объектам в кадре
                    if ind in wear_tracks.index:
                        helmet_id, helmet_x, helmet_y, helmet_w, helmet_h  = wear_tracks.loc[ind]['id': 'height']
                        insec_ratio = intersection_ratio(human_x, human_y, human_w, human_h, helmet_x, helmet_y, helmet_w, helmet_h, wear)
                        pairs.loc[human_id][helmet_id].append(insec_ratio)

        for i in actual_human_id:
            for c in wear_id_list:
                if pairs.loc[i][c]:
                    mean_insec_ratio = np.array(pairs.loc[i][c]).mean()
                    if len(pairs.loc[i][c]) > 15 and mean_insec_ratio > 0.7:           # если среднее значение intersection_ratio больше порога, ставим плюсик в соответствующую ячейку таблицы учёта log_book
                        log_book.loc[i][wear] = '+'
                    #     print(f'    human [{i}] associates with {wear} [{c}]:      mean_insec_ratio: {mean_insec_ratio:.2f}')
                    # else: print(f'                                        human [{i}] --------------------------- with {wear} [{c}]:      mean_insec_ratio: {mean_insec_ratio:.2f}')
        return log_book

In [ ]:
def get_vidpath_from_filepath(filepath):
    exp_dir = dirname(dirname(filepath))
    file_number = get_f_number(filepath)
    vid_file_name = file_number + '.mp4'
    vid_file_path = os.path.join(exp_dir, vid_file_name)
    return vid_file_path

In [ ]:
for key, filepath in od.items():
    file_number = get_f_number(filepath)
    try:
        with open(filepath, 'r') as f:
            tracks = f.readlines()
        tracks = [t.strip().split() for t in tracks]
        tracks = np.array(tracks).astype('int')
        all_classes_tracks = pd.DataFrame(data=tracks, columns=['Frame', 'id', 'x', 'y', 'width', 'height', 'Class'])
        human_mask = all_classes_tracks['Class']==0
        human_tracks = all_classes_tracks[human_mask]           # Людей лучше трекать отдельно! На сколько лучше, скзать не могу. На имеющейся тестовой выборке разница в 1 непосчитанном человеке.
        detections = True

        frame_dict = {k:list() for k in all_classes_tracks['Frame'].unique()}               # ключи словаря - номера кадров, значения - индексы строк с таким номером кадра
        for i in all_classes_tracks.index:
            frame_dict[all_classes_tracks.loc[i]['Frame']].append(i)

    except:
        print(f'Video {file_number}: No detections')
        print()
        detections = False

    if detections:
        vid_file_path = get_vidpath_from_filepath(filepath)
        vidcap = cv2.VideoCapture(vid_file_path, )
        success, image = vidcap.read()

        gate_y = 375 * image.shape[0] // 640    # примерная координата турникета
        delta = 50 * image.shape[0] // 640      # эмпирическая поправка (она вроде не особо влияет, но пусть будет)
        gate_y -= delta
        res = image.shape[0]                    # размер видео по вертикали

        human_id_list = list(human_tracks['id'].unique())
        n_humans = len(human_id_list)

        new_id_dict = {k: i+1 for i, k in enumerate(human_id_list)}     # перенумерация id-шников с 1 до n_humans
        for i in human_tracks.index:
            id = human_tracks.loc[i]['id']
            human_tracks.loc[i]['id'] = new_id_dict[id]

        human_id_list = list(range(1, n_humans+1))

        bb_y = {id:list() for id  in human_id_list}               # координата 'у' верхнего левого угла
        bb_h = {id:list() for id  in human_id_list}               # высота баундин бокса
        bb_frame = {id:list() for id  in human_id_list}           # номер кадра
        bb_start = {id:list() for id  in human_id_list}              # с какой стороны заходит (= 0, если чел выходит,  = res, если заходит)

        '''
        Предобработка треков.
        Иногда один и тот же id сначала проходит в одном направлении, а потом в другом, нужно посчитать его как одного входящего и одного выходящего.
        В данной реализации сравнивается первая и последняя координата объекта, поэтому если он сначала вошёл, а потом вышел, начальная и последняя координаты будут по одну сторону от турникета и алгоритм не сработает.
        Идея системы коррекции: после того как объект прошёл коридор - все дальнейшие id этого объекта заменяются на новые, и при последующем проходе, он будет считаться как новый объект с новым id.
        '''

        for i in human_tracks.index:
            id = human_tracks.loc[i]['id']
            frame_n = human_tracks.loc[i]['Frame']
            x, y, w, h = human_tracks.loc[i]['x':'height']
            bb_y[id].append(y)
            bb_h[id].append(h)
            bb_frame[id].append(frame_n)
            if i == 0:
                bb_start[id] = bb_y[id][0]
            else:
                if bb_y[id][-1] < bb_y[id][0]:
                    bb_start[id] = res
                else:
                    bb_start[id] = 0

            if len(bb_frame[id]) > 20:

                condition_in = bb_start[id] == 0 and (bb_y[id][-1] + bb_h[id][-1]) / res > 0.75 and bb_frame[id][-1] - bb_frame[id][-2] > 20          # условие смены id при проходе внутрь: 1) стартовая точка наверху, 2) нижняя граница бб пересекла порог (0,75 от высоты изображения) 3) бб не детектировался в течение 10 предыдущих кадров                                         # go in
                condition_out = bb_start[id] == res and bb_y[id][-1] / res < 0.35 and bb_frame[id][-1] - bb_frame[id][-2] > 20                        # условие смены id при проходе наружу: 1) стартовая точка внизу, 2) верхняя граница бб пересекла порог (0,35 от высоты изображения) 3) бб не детектировался в течение 10 предыдущих кадров

                if condition_in or condition_out:
                    # print(f'New human {id} -> {n_humans+1}, frame: {frame_n},   i = {i}')
                    # print()
                    n_humans += 1
                    human_id_list.append(n_humans)                  # добавить новую id в список
                    bb_y[n_humans] = []
                    bb_h[n_humans] = []
                    bb_frame[n_humans] = []
                    bb_start[n_humans] = [None]
                    current_position = list(human_tracks.index).index(i)
                    for j in human_tracks.index[current_position:]:       # во всех последующих детекциях данный id меняются новый
                        if human_tracks.loc[j]['id'] == id:
                            human_tracks.loc[j]['id'] = n_humans
                    # print()
                    # print()
        human_id_list

        '''
        Собственно подсчёт.
        Если условная координата "y" турникета находится в интервале между начальной и конечной координатой объекта, то защитывается проход
        '''

        came_in = 0                                         # количество вошедших
        came_out = 0                                        # количество вышедших
        log_book = pd.DataFrame(columns=['direction', 'helmet', 'uniform'], index=human_id_list, data='-')          # Журнал учёта 
        log_book.index.names = ['id']

        human_point = {id:list() for id in human_id_list}
        actual_human_id = []                                # сюда будем добавлять посчитанных людей

        print(f'Track {file_number}):')

        for i in human_tracks.index:
            id = human_tracks.loc[i]['id']
            frame_n = human_tracks.loc[i]['Frame']
            x, y, w, h = human_tracks.loc[i]['x':'height']
            box_center = x + w//2, y + h//2
            human_point[id].append(box_center[1])               # оставляем только координату Y

        for id, h in human_point.items():
            if h and h[0] < gate_y  < h[-1]:
                came_in += 1
                actual_human_id.append(id)
                log_book.loc[id]['direction'] = 'came in'
                # print(f'          Human {id} came in')

            elif h and h[0] > gate_y > h[-1]:
                came_out += 1
                actual_human_id.append(id)
                log_book.loc[id]['direction'] = 'came out'
                # print(f'          Human {id} came out')
            
            else:
                log_book = log_book.drop([id], axis=0)

        print()
        print(f'    {came_in} humans came in')
        print(f'    {came_out} humans came out')
        print()

        log_book = check_work_wear('helmet')                # проверка шлемов
        log_book = check_work_wear('uniform')               # проверка жилетов

        display(log_book)


        print()
        print()

Track 1):

    3 humans came in
    0 humans came out



,direction,helmet,uniform
id,,,
1,came in,+,+
2,came in,+,+
4,came in,+,+




Track 2):

    3 humans came in
    0 humans came out



,direction,helmet,uniform
id,,,
1,came in,+,+
4,came in,+,+
5,came in,+,+




Track 3):

    2 humans came in
    0 humans came out



...

,direction,helmet,uniform
id,,,
1,came out,+,+




Track 40):

    0 humans came in
    1 humans came out



,direction,helmet,uniform
id,,,
1,came out,-,-




Track 42):

    2 humans came in
    0 humans came out



,direction,helmet,uniform
id,,,
1,came in,-,-
2,came in,-,-


In [ ]:
# Ошибки:

# Видео 3: не посчитан 1 человек (в изолированном алгоритме подсчёт происходит верно, что связано с более устойчивым присвоением id)
# Видео 13: посчитан лишний жилет (куртка очень похожа)
# Видео 14: посчитан лишний человек (плюс жилет на нём) т.к. на нём детектится 2 бб. Не посчитан 1 шлем.
# Видео 17: не посчитан 1 синий шлем - отсутствует детекция
# Видео 30: не посчитаны 2 выходящих в жилетах (плохая детекция) 

In [ ]:
go_in_true = 3+3+3+4+1+1+2+2+4+1+1+1+2+1+1+1+1+7+2
go_out_true = 4+4+4+1+2+3+4+4+3+4+6+3+1+1+1+1+1+3+2+6+1+1
go_in_err = 1
go_out_err = 1+2

err = (go_in_err + go_out_err)/(go_in_true + go_out_true)
f'Точность подсчёта людей: {1 - err:.2f}'

'Точность подсчёта людей: 0.96'

In [ ]:
helmets_true = 3+3+2+3+1+1+1+1+3+1+1+4+1+1+1+1+2+4+1
helmets_err = 1+1

err = helmets_err / helmets_true
f'Точность подсчёта касок: {1 - err:.2f}'

'Точность подсчёта касок: 0.94'

In [ ]:
uniform_true = 3+3+2+3+1+2+2+2+1+1+4+3+3+3+2+1+1+1+1+1+3+2+4+1
uniform_err = 1+1+2

err = uniform_err / uniform_true
f'Точность подсчёта жилетов: {1 - err:.2f}'

'Точность подсчёта жилетов: 0.92'

In [ ]:
helmet_tp = helmets_true - 2
helmet_fp = 0
helmet_precision = helmet_tp / (helmet_tp + helmet_fp)
helmet_recall = helmet_tp / helmets_true
f'precision: {helmet_precision:.2f},        recall: {helmet_recall:.2f}'

'precision: 1.00,        recall: 0.94'

In [ ]:
uniform_tp = uniform_true - 2
uniform_fp = 0
uniform_precision = uniform_tp / (uniform_tp + uniform_fp)
uniform_recall = uniform_tp / uniform_true
f'precision: {uniform_precision:.2f},        recall: {uniform_recall:.2f}'

'precision: 1.00,        recall: 0.96'